In [58]:
from datetime import datetime, timedelta
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

In [50]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    start="2021-04-01",
    end="2021-12-01",
    tickers="NDX",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [ ]:
def get_yf_data(tickers: str, start_str: datetime, end_str: datetime):
    return yf.download(
        tickers=tickers,
        start=start_str,
        end=end_str,
        interval="1d",
        group_by="ticker",
    ).dropna()

In [46]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]

In [81]:
class My_Strategy(Strategy):
    # RSI
    prop_rsi = 35  # 14
    prop_rsi_buy = 60  # 70
    prop_rsi_sell = 50  # 30

    take_profit = 2
    stop_loss = 2

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def next(self):
        # 計算できていない場合トレードしない
        if len(self.data.index) < 35:  # MACD
            return

        # MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            if self.position.is_short:
                self.position.close()

            if self.position.is_long or self.prop_rsi_buy > self.rsi[-1]:
                return

            self.buy(
                sl=self.data.Close[-1] * (1 - (self.stop_loss / 100)),
                tp=self.data.Close[-1] * (1 + (self.take_profit / 100)),
            )
            return

        # MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            if self.position.is_long:
                self.position.close()

            if self.position.is_short or self.rsi[-1] > self.prop_rsi_sell:
                return

            self.sell(
                sl=self.data.Close[-1] * (1 + (self.stop_loss / 100)),
                tp=self.data.Close[-1] * (1 - (self.take_profit / 100)),
            )
            return

In [83]:
start_at = datetime.strptime("2018/05/01 00:00:00", "%Y/%m/%d %H:%M:%S")
end_at = datetime.strptime("2020/12/31 00:00:00", "%Y/%m/%d %H:%M:%S")

bt = Backtest(
    get_yf_data("NDX", start_at, end_at),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)


# 最適化
optimize = bt.optimize(
    prop_rsi_buy=range(10, 90, 5),
    prop_rsi_sell=range(10, 90, 5),
    take_profit=range(2, 8, 2),
    stop_loss=range(2, 8, 2),
    # constraint=lambda p: p.prop_rsi_sell < p.prop_rsi_buy,
    maximize="SQN",
)
bt.plot()
print(optimize)
print(optimize._strategy)


# 最適化したものが通用するのか、違う期間で検証する。
bt = Backtest(
    get_yf_data("NDX", end_at, end_at + (end_at - start_at)),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)

optimize = bt.optimize(
    prop_rsi_buy=optimize._strategy.prop_rsi_buy,
    prop_rsi_sell=optimize._strategy.prop_rsi_sell,
    stop_loss=optimize._strategy.stop_loss,
    take_profit=optimize._strategy.take_profit,
)
bt.plot()
print(optimize)
print(optimize._strategy)

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 2304 configurations.
  output = _optimize_grid()


Start                     2018-05-01 00:00:00
End                       2020-12-30 00:00:00
Duration                    974 days 00:00:00
Exposure Time [%]                       21.25
Equity Final [$]                  11443302.10
Equity Peak [$]                   11544135.03
Return [%]                              14.43
Buy & Hold Return [%]                   92.24
Return (Ann.) [%]                        5.18
Volatility (Ann.) [%]                    9.67
Sharpe Ratio                             0.54
Sortino Ratio                            0.91
Calmar Ratio                             0.93
Max. Drawdown [%]                       -5.57
Avg. Drawdown [%]                       -1.83
Max. Drawdown Duration      222 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                   26
Win Rate [%]                            65.38
Best Trade [%]                           4.88
Worst Trade [%]                         -3.00
Avg. Trade [%]                    

Start                     2020-12-31 00:00:00
End                       2022-11-11 00:00:00
Duration                    680 days 00:00:00
Exposure Time [%]                       12.53
Equity Final [$]                   9266721.52
Equity Peak [$]                   10502357.67
Return [%]                              -7.33
Buy & Hold Return [%]                   -8.31
Return (Ann.) [%]                       -3.99
Volatility (Ann.) [%]                    7.09
Sharpe Ratio                             0.00
Sortino Ratio                            0.00
Calmar Ratio                             0.00
Max. Drawdown [%]                      -13.25
Avg. Drawdown [%]                       -2.92
Max. Drawdown Duration      319 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                   10
Win Rate [%]                            60.00
Best Trade [%]                           2.26
Worst Trade [%]                         -6.68
Avg. Trade [%]                    